**Engy-4390** Nuclear Systems Design and Analysis, UMass Lowell, Chem. Eng. Nuclear Program Fall 2024; Prof. V. F. de Almeida

# Constant Thermal Conductivity, Constant Source, Robin BC

**Student(s):**  Kyle Mills and Billy Manco and Donaita Anglade <br>
**Course:** Engy-4390: Nuclear Systems Design and Analysis  <br>
**Mentor and Collaborator:** Subash Sharma

UMass Lowell, Dept. of Chemical Engineering, Nuclear Energy Program 
<br>
**21 Oct 2024**

---
## Table of Contents<a id="toc"></a>
* [Definitions](#defs)
* [Setup](#setup)
* [Test](#test)
---

## [Definition of the necessary functions](#toc)<a id="defs"></a>
The fourth test is done with Robin BC and a position-dependent source bias. The conditions for this test are stated below,
* k(x) = 87219  [$ \frac{W}{mK} $]
* f(x) = 166117  [$ \frac{W}{m^3} $]
* T(a) = 20 [°C], and T(b) = 40 [°C]

## [Setup](#toc)<a id="setup"></a>

In [6]:
'''Generate the target function f'''
import matplotlib.pyplot as plt
import math as m
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
from scipy.integrate import quad

In [7]:
'''Domain'''
'''Rayleigh Ritz Method with Lagrange FE Basis Functions'''
x_min=0
x_max=0.05

degree = 2  #Degree of the lagrange polinomials used
n_elem = 10 #Number of elements used

In [8]:
'''Parent mapping'''
from pyfires import get_parent_mapping

In [9]:
'''The basis functions'''
from pyfires import get_parent_basis_functions

In [10]:
'''Again the function for the obtaining of the basis functions'''
from pyfires import global_basis_function

In [11]:
'''All global basis functions'''
from pyfires import get_global_basis_functions

In [12]:
'''Pedagogical inner product'''
from pyfires import inner_product

In [13]:
'''FEM Solution'''
'''Domain partition'''
def get_domain_partition(degree, n_elem, x_min, x_max, bc_x_min='flux', bc_x_max='flux'):
    #assert degree == 1
    # Local node numbering on parent domain
    # --0--------------1---->
    #  -1      0      +1    zetta
    gnodes_x = np.linspace(x_min, x_max, n_elem*degree+1, dtype=np.float64)
    patches = list()
    local_to_global_node_id_map = list()
    for e in range(n_elem):
        gnode_id_1 = degree*e   #left
        gnode_id_2 = degree*e+1 #center
        gnode_id_3 = degree*e+2 #right
        x1 = gnodes_x[gnode_id_1]
        x2 = gnodes_x[gnode_id_2]
        x3 = gnodes_x[gnode_id_3]
        # Local node id:  0   1
        patches.append((x1, x2, x3))
        # Local node id:                        0           2             1
        local_to_global_node_id_map.append([gnode_id_1, gnode_id_2,  gnode_id_3])
    if bc_x_min == 'essential':
        local_to_global_node_id_map[0][0] = -1
    if bc_x_max == 'essential':
        local_to_global_node_id_map[-1][-1] = -1
    return (patches, gnodes_x, local_to_global_node_id_map)

domain_partition = get_domain_partition(degree, n_elem, x_min, x_max)
parent_mapping = get_parent_mapping()
parent_basis_functions = get_parent_basis_functions()

phi_list = get_global_basis_functions(domain_partition, parent_mapping, parent_basis_functions, global_basis_function)[0]
phi_prime_list = get_global_basis_functions(domain_partition, parent_mapping, parent_basis_functions, global_basis_function)[1]

In [14]:
'''Thermal conductivity'''
from pyfires import constant_thermal

k_cond = constant_thermal()

In [15]:
'''Heat Generation Density'''
from pyfires import heat_gen_pts 

func_x = heat_gen_pts()

## [Test](#toc)<a id="test"></a>

In [17]:
'''Parameters'''
htc = 10      #Heat transfer coefficient

In [18]:
'''Thermal conductivity'''
#Remove the comment if you would like to see the graph
'''
plt.figure()
plt.plot(np.linspace(0,0.05,200),k_cond(np.linspace(0,0.05,200)),label='Target Function')
plt.title('Thermal Conductivity') 
plt.xlabel("Position along the width of the channel wall [m]")
plt.ylabel("K [W/mK]") 
plt.grid()
plt.show()
'''

'\nplt.figure()\nplt.plot(np.linspace(0,0.05,200),k_cond(np.linspace(0,0.05,200)),label=\'Target Function\')\nplt.title(\'Thermal Conductivity\') \nplt.xlabel("Position along the width of the channel wall [m]")\nplt.ylabel("K [W/mK]") \nplt.grid()\nplt.show()\n'

In [19]:
'''Source bias'''
#Remove the comment if you would like to see the graph
'''
plt.figure()
plt.plot(np.linspace(0,0.05,200),func_x(np.linspace(0,0.05,200)),label='Target Function')
plt.title('Values for f(x)') 
plt.xlabel("Position along the width of the channel wall [m]") 
plt.ylabel("Heat Generation Density [W/m^3]") 
plt.grid()
plt.show()
'''

'\nplt.figure()\nplt.plot(np.linspace(0,0.05,200),func_x(np.linspace(0,0.05,200)),label=\'Target Function\')\nplt.title(\'Values for f(x)\') \nplt.xlabel("Position along the width of the channel wall [m]") \nplt.ylabel("Heat Generation Density [W/m^3]") \nplt.grid()\nplt.show()\n'

In [20]:
'''Building the slope function'''
from pyfires import slope_func

temp_func = slope_func()
slope_func = slope_func()

plt.figure()
plt.plot(np.linspace(0,0.05,200),temp_func(np.linspace(0,0.05,200)))
plt.title("Change in temperature across the domain")
plt.xlabel("Position along the width of the channel [m]") 
plt.ylabel("Temperature [C]") 
plt.grid()
plt.show()

plt.figure()
plt.plot(np.linspace(0,0.05,200),slope_func(np.linspace(0,0.05,200)))
plt.title("Derivative of Heat Generation Source") 
plt.xlabel("Position along the width of the channel [m]")
plt.ylabel("Heat flux[C/m]")
plt.grid()
plt.show()

TypeError: slope_func() missing 2 required positional arguments: 'x_min' and 'x_max'

In [ ]:
'''FEM Solution'''
'''Domain partition'''
def get_domain_partition(degree, n_elem, x_min, x_max, bc_x_min='flux', bc_x_max='flux'):
    #assert degree == 1
    # Local node numbering on parent domain
    # --0--------------1---->
    #  -1      0      +1    zetta
    gnodes_x = np.linspace(x_min, x_max, n_elem*degree+1, dtype=np.float64)
    patches = list()
    local_to_global_node_id_map = list()
    for e in range(n_elem):
        gnode_id_1 = degree*e   #left
        gnode_id_2 = degree*e+1 #center
        gnode_id_3 = degree*e+2 #right
        x1 = gnodes_x[gnode_id_1]
        x2 = gnodes_x[gnode_id_2]
        x3 = gnodes_x[gnode_id_3]
        # Local node id:  0   1
        patches.append((x1, x2, x3))
        # Local node id:                        0           2             1
        local_to_global_node_id_map.append([gnode_id_1, gnode_id_2,  gnode_id_3])
    if bc_x_min == 'essential':
        local_to_global_node_id_map[0][0] = -1
    if bc_x_max == 'essential':
        local_to_global_node_id_map[-1][-1] = -1
    return (patches, gnodes_x, local_to_global_node_id_map)

domain_partition = get_domain_partition(degree, n_elem, x_min, x_max)
parent_mapping = get_parent_mapping()
parent_basis_functions = get_parent_basis_functions()

phi_list = get_global_basis_functions(domain_partition, parent_mapping, parent_basis_functions, global_basis_function)[0]
phi_prime_list = get_global_basis_functions(domain_partition, parent_mapping, parent_basis_functions, global_basis_function)[1]

In [ ]:
#Building of the A matrix
A_mtrx = np.zeros((len(phi_list), len(phi_list)), dtype=np.float64)
patches = domain_partition[0]
for i in range(len(phi_list)):
    for j in range(len(phi_list)):
        
        phi_i=phi_list[i]
        phi_j=phi_list[j]
        
        phi_prime_i=phi_prime_list[i]
        phi_prime_j=phi_prime_list[j]
        
        h_e=(x_max-x_min)/n_elem
        
        d_x_phi_prime_j = lambda x: k_cond(x) * ((2/h_e)*phi_prime_j(x))
        
        prima = lambda x: phi_prime_i(x)*(2/h_e)
        
        A_mtrx[i,j] = inner_product(prima, d_x_phi_prime_j, patches) + htc*phi_i(x_min)*phi_j(x_min) + htc*phi_i(x_max)*phi_j(x_max)

In [ ]:
'''Build load vector'''
#The load vector
b_vec = np.zeros(len(phi_list), dtype=np.float64)
patches = domain_partition[0]

for i in range(len(phi_list)):
    phi_i=phi_list[i]
    phi_prime_i=phi_prime_list[i]
    
    b_vec[i] = inner_product(temp_func, phi_i, patches)-htc*(temp_func(x_max)-u_b)*phi_i(x_max)-htc*(temp_func(x_min)-u_a)*phi_i(x_min)
    
    first_term = lambda x: slope_func(x)*k_cond(x)
    phi_prima_i = lambda x: phi_prime_i(x)*(2/h_e)
    
    b_vec[i] -= inner_product(first_term, phi_prima_i, patches)

In [ ]:
'''Solution'''
'''Compute optimal coefficient vector'''
c_star_vec = np.linalg.solve(A_mtrx, b_vec)

'''Plot comparison of f and g_best_vec'''
def u_star(x):
    g_x=temp_func(x)
    for (j,phi_i) in enumerate(phi_list):
        g_x=g_x+(c_star_vec[j])*phi_i(x)
    return g_x

def u_star_prime(x):
    g_x=slope_func(x)
    for j in range(len(phi_list)):
        g_x=g_x+(c_star_vec[j])*((2/h_e)*phi_prime_list[j](x))
    return g_x

plt.figure()
plt.plot(np.linspace(x_min,x_max,2000),u_star(np.linspace(x_min,x_max,2000)),label='Rayleigh-Ritz solution')

#plt.plot(np.linspace(x_min,x_max,2000),sol(np.linspace(x_min,x_max,2000)),label='Analytic solution')
plt.title('Solutions with Dirichlet BC, constant source and conductivity')
plt.xlabel("Position aling the width of the channel wall [m]")
plt.ylabel("Temperature [°C]")
plt.legend()
plt.grid()
plt.show()

print('The value of the function at the leftmost end of the domain is: %4.2f [°C].'%u_star(x_min))
print('The value of the function at the rightmost end of the domain is: %4.2f [°C].'%u_star(x_max))

#Plotting the flux
plt.figure()
plt.plot(np.linspace(x_min,x_max,2000),-k_cond(np.linspace(x_min,x_max,2000))*u_star_prime((np.linspace(x_min,x_max,2000))),label='Rayleigh-Ritz solution')
plt.xlabel("Position along the width of the channel wall [m]")
plt.ylabel("Heat flux [W/m2]")
plt.legend()
plt.grid()
plt.show()

'''Flux at boundary'''
print('q_na computed = ', -k_cond(x_min)*u_star_prime(x_min))
print('q_nb computed = ', -k_cond(x_max)*u_star_prime(x_max))

In [ ]:
'''Solution'''
'''Compute optimal coefficient vector'''
c_star_vec = np.linalg.solve(A_mtrx, b_vec)

plt.figure()
plt.plot(np.linspace(x_min,x_max,2000),u_star(np.linspace(x_min,x_max,2000)),label='Rayleigh-Ritz solution')
#plt.plot(np.linspace(x_min,x_max,2000),sol(np.linspace(x_min,x_max,2000)),label='Analytic solution')
plt.xlabel("Position aling the width of the channel wall [m]")
plt.ylabel("Temperature [°C]")
plt.title('Solutions with Robin BC, variable source')
plt.legend()
plt.grid()
plt.show()

print('The value of the function at the leftmost end of the domain is: %4.2f [°C].'%u_star(x_min))
print('The value of the function at the rightmost end of the domain is: %4.2f [°C].'%u_star(x_max))


#Plotting the flux
plt.figure()
plt.plot(np.linspace(x_min,x_max,2000),-k_cond(np.linspace(x_min,x_max,2000))*u_star_prime((np.linspace(x_min,x_max,2000))),label='Rayleigh Ritz solution')
plt.xlabel("Position along the width of the channel wall [m]")
plt.ylabel("Heat flux [W/m2]")
plt.legend()
plt.title('Solutions with Robin BC, variable source')
plt.grid()
plt.show()

#print(sol(np.linspace(x_min,x_max,2000)))

'''Flux at boundary'''
print('q_na computed = ', -k_cond(x_min)*u_star_prime(x_min))
print('q_nb computed = ', -k_cond(x_max)*u_star_prime(x_max))
#print('q_nb given    = ', q_nb)
#print('flux error [%]= ',(-f_cond(x_max)*g_prime(x_max) - q_nb)/q_nb*100)